In [50]:
import os
import json
import pandas as pd
import traceback

from ai21 import AI21Client
from langchain_ai21 import ChatAI21
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks.base import BaseCallbackManager, BaseCallbackHandler
import PyPDF2

In [85]:
os.environ["AI21_API_KEY"] = os.getenv("AI21_API_KEY")
llm = ChatAI21(model = "jamba-instruct-preview", max_tokens = 1024,temperature = 0.5, callbacks = BaseCallbackManager(handlers = [BaseCallbackHandler]))

In [86]:
TEMPLATE = """
    Text: {text}
    You are an expert MQC maker. Given the above text, it is your job \
    to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
    Make sure the questions are not repeated and check all the questions to be conforming the text as well.
    Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
    Ensure to make {number} MCQs.
    ### RESPONSE_JSON
    {response_json} 
"""

In [87]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [88]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [89]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

In [90]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [91]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template = TEMPLATE2)

In [92]:
quiz_evaluation_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [93]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, quiz_evaluation_chain],
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    output_variables = ["quiz", "review"],
    verbose = True
)

In [94]:
file_path = r"D:\Projects\MCQ Generator\data.txt"
with open(file_path, "r") as file:
    TEXT = file.read().strip()

In [95]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [126]:
NUMBER = 5
SUBJECT = "biology"
TONE = "complex"

In [127]:
response = generate_evaluate_chain(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each d

In [129]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [130]:
quiz = response.get("quiz")

In [131]:
review = response.get("review")

In [132]:
quiz = json.loads(quiz[8:-4])

In [133]:
quiz

{'1': {'mcq': 'What are the four unifying themes in biology?',
  'options': {'a': 'Cell biology, evolution, energy processing, and homeostasis',
   'b': 'Cell biology, genetics, ecology, and physiology',
   'c': 'Molecular biology, anatomy, physiology, and evolution',
   'd': 'Cell biology, genetics, ecology, and physiology'},
  'correct': 'a'},
 '2': {'mcq': 'What is the primary unit of life?',
  'options': {'a': 'Molecules',
   'b': 'Cells',
   'c': 'Organelles',
   'd': 'Organisms'},
  'correct': 'b'},
 '3': {'mcq': 'Which process explains the unity and diversity of life?',
  'options': {'a': 'Cell division',
   'b': 'Evolution',
   'c': 'Photosynthesis',
   'd': 'Cell respiration'},
  'correct': 'b'},
 '4': {'mcq': 'What do all organisms have in common?',
  'options': {'a': 'Ability to regulate internal environment',
   'b': 'Presence of genes',
   'c': 'Use of energy',
   'd': 'All of the above'},
  'correct': 'd'},
 '5': {'mcq': 'What is the study of the molecular biology of a ce

In [134]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [135]:
quiz = pd.DataFrame(quiz_table_data)

In [136]:
quiz.to_csv("biology_quiz.csv", index = False)